### Module Imports

In [ ]:
import os
import numpy as np
import cv2

from tqdm import tqdm

In [4]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from torch.utils.data import DataLoader

### Creating dataloaders from the dataset

In [5]:
IMG_SIZE = 28

In [6]:
training_dataset_path = 'new_data/training_data/'
testing_dataset_path = 'new_data/testing_data/'

In [7]:
characters = os.listdir(training_dataset_path)

# characters

In [8]:
labels_dict = dict()
for each in range(len(characters)):
    labels_dict[characters[each]] = each
    
# labels_dict

In [9]:
num_labels = len(characters)
num_labels

36

In [10]:
def data_compiler(dataset_path):
    dataset = []

    for character in characters:
        character_path = os.path.join(dataset_path, character)
        # print(character_path)

        for character_image in os.listdir(character_path):
            image_path = os.path.join(character_path, character_image)

            img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            # resizing the image
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            # plt.imshow(img)
            # converting to tensor
            img = torch.tensor(img).view(-1, IMG_SIZE*IMG_SIZE)
            # scaling the data
            img = (img/255.0)

            # print(img.shape)
            dataset.append([img, np.eye(num_labels)[labels_dict[character]]])
            
    return dataset


In [11]:
training_dataset = data_compiler(training_dataset_path)
len(training_dataset)

20628

In [12]:
testing_dataset = data_compiler(testing_dataset_path)
len(testing_dataset)

1008

In [13]:
BATCH_SIZE = 32

In [14]:
train_dataloader = DataLoader(training_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(testing_dataset, batch_size=BATCH_SIZE, shuffle=True)

### Training CNN

In [16]:
class ConvNet(nn.Module):
    def __init__(self, num_labels, img_size=IMG_SIZE):
        super().__init__()
        self.num_labels = num_labels
        self.img_size = img_size
        
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        
        x = torch.rand(self.img_size, self.img_size).view(-1,1,self.img_size,self.img_size)
        self._to_linear = None
        self.convs(x)
        
        self.fc1 = nn.Linear(self._to_linear, 256)
        self.fc2 = nn.Linear(256, self.num_labels)
        
    def convs(self, x):
        x = x.view(-1,1,self.img_size,self.img_size)
        x = F.max_pool2d(F.relu(self.conv1(x)), kernel_size=2)
        x = F.max_pool2d(F.relu(self.conv2(x)), kernel_size=2)
        x = F.max_pool2d(F.relu(self.conv3(x)), kernel_size=2)

        if self._to_linear is None:
            self._to_linear = 1
            for each in range(len(x[0].shape)):
                self._to_linear *= x[0].shape[each]
        
        return x
    
    def forward(self, x):
        x = self.convs(x)
        x = x.view(-1, self._to_linear)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        
        return x
    
    def save(self, path='best_model.pth'):
        torch.save(self.state_dict(), path)

    def load(self, path='best_model.pth'):
        self.load_state_dict(torch.load(path))
        self.eval()
        
    def fit(self, train_loader, optimizer=Adam, loss_function=CrossEntropyLoss(), learning_rate=0.001, epochs=5):
        min_loss = float('inf')
        self.optimizer = optimizer(self.parameters(), lr=learning_rate)
        self.loss_func = loss_function
        self.epochs = epochs
        
        for epoch in tqdm(range(self.epochs)):
            for batch in train_loader:
                train_X, train_y = batch
                pred_train_y = self.forward(train_X)
                
                loss = self.loss_func(pred_train_y, train_y)
                
                # self.zero_grad()
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()
            
            print(f'epoch={epoch+1}: loss={loss}')
            # if loss < min_loss:
            #     self.save()
            #     min_loss = loss

        self.save()
        
    def test(self, test_loader):
        self.load()
        
        total = len(test_loader.dataset)
        correct = 0
        
        for batch in test_loader:
            test_X, test_y = batch
            pred_test_y = self.forward(test_X)

            for each in range(len(test_y)):
                pred = torch.argmax(pred_test_y, 1)[each]
                true = torch.argmax(test_y, 1)[each]
                if pred == true:
                    correct += 1
                    
        print(f'accuracy: {correct/total}')

In [17]:
model = ConvNet(num_labels)

In [18]:
model

ConvNet(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=1152, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=36, bias=True)
)

In [19]:
model.fit(train_dataloader, learning_rate=0.001, epochs=10)

 10%|████▍                                       | 1/10 [00:08<01:14,  8.31s/it]

epoch=1: loss=0.2724030649596898


 20%|████████▊                                   | 2/10 [00:16<01:06,  8.37s/it]

epoch=2: loss=0.036471910066134684


 30%|█████████████▏                              | 3/10 [00:25<00:58,  8.35s/it]

...

epoch=8: loss=0.020000559640224225


 90%|███████████████████████████████████████▌    | 9/10 [01:15<00:08,  8.38s/it]

epoch=9: loss=0.07627599721255648


100%|███████████████████████████████████████████| 10/10 [01:23<00:00,  8.37s/it]

epoch=10: loss=0.00034006594659850007


In [20]:
model.test(test_dataloader)

accuracy: 0.9841269841269841
